In [11]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf


dataframe = spark.read.csv('/Users/Katherine/Downloads/creditcard.csv', header=True, inferSchema=True)
dataframe.cache()

DataFrame[Time: decimal(10,0), V1: double, V2: double, V3: double, V4: double, V5: double, V6: double, V7: double, V8: double, V9: double, V10: double, V11: double, V12: double, V13: double, V14: double, V15: double, V16: double, V17: double, V18: double, V19: double, V20: double, V21: double, V22: double, V23: double, V24: double, V25: double, V26: double, V27: double, V28: double, Amount: double, Class: int]

In [3]:
dataframe.show()

76340557365201|   -1.55973769907953|  0.160841747266769|  1.23308974041888|  0.345172827050629| 0.917229867699146| 0.970116716069048| -0.266567764915222| -0.479129929276704| -0.526608502569153|   0.47200411177674|  -0.725480944982201|  0.075081351540202| -0.406866573198217|  -2.19684802485647|  -0.503600328973703|  0.984459785590244|   2.45858857639219| 0.0421188969891572| -0.481630823956716| -0.621272013713977|   0.392053289557744|  0.949594245504846|  46.8|    0|
|  15|   1.4929359769862|  -1.02934573189487|  0.45479473374366|  -1.43802587991702|  -1.55543410136344| -0.720961147043557|   -1.08066413038614|-0.0531271179483221|  -1.9786815953872|   1.63807603690446|  1.07754241162743| -0.63204651464934|  -0.41695716661602| 0.0520105153724404|-0.0429789228232019| -0.166432496451972|   0.304241418614353|  0.554432499062278| 0.0542295152184719| -0.387910172646258|  -0.177649846438814| -0.175073809074822| 0.0400022190621329|  0.295813862676508|   0.33293059939425| -0.220384850672322|  0.02

In [10]:
dataframe.printSchema()

root
 |-- Time: decimal(10,0) (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double

In [8]:
dataframe.groupBy('Class').count().show()
# this shows that the dataset is imbalanced 

+-----+------+
|Class| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



In [29]:
amountNormalized = dataframe.select('Amount')
amountNormalized.show()
print type(amountNormalized)
amountNormalized.cache()


+------+
|Amount|
+------+
|149.62|
|  2.69|
|378.66|
| 123.5|
| 69.99|
|  3.67|
|  4.99|
|  40.8|
|  93.2|
|  3.68|
|   7.8|
|  9.99|
| 121.5|
|  27.5|
|  58.8|
| 15.99|
| 12.99|
|  0.89|
|  46.8|
|   5.0|
+------+
only showing top 20 rows

<class 'pyspark.sql.dataframe.DataFrame'>


DataFrame[Amount: double]

In [43]:
slen = udf(lambda vs: Vectors.dense(vs), VectorUDT())
# amountNormalized.select('*', slen(amountNormalized['Amount']).alias('slen')).show()
dataframe_with = dataframe.withColumn('slen', slen(amountNormalized['Amount']))
dataframe_with.cache()
dataframe_with.show()
mmScaler = MinMaxScaler(inputCol="slen", outputCol="scaled")
model = mmScaler.fit(dataframe_with)
print mmScaler.explainParams()
model.transform(dataframe_with).show()



+------+--------+
|Amount|    slen|
+------+--------+
|149.62|[149.62]|
|  2.69|  [2.69]|
|378.66|[378.66]|
| 123.5| [123.5]|
| 69.99| [69.99]|
|  3.67|  [3.67]|
|  4.99|  [4.99]|
|  40.8|  [40.8]|
|  93.2|  [93.2]|
|  3.68|  [3.68]|
|   7.8|   [7.8]|
|  9.99|  [9.99]|
| 121.5| [121.5]|
|  27.5|  [27.5]|
|  58.8|  [58.8]|
| 15.99| [15.99]|
| 12.99| [12.99]|
|  0.89|  [0.89]|
|  46.8|  [46.8]|
|   5.0|   [5.0]|
+------+--------+
only showing top 20 rows



In [44]:
# TO SHOW ONLY AMOUNT COLUMN SCALER
amountNormalized_with = amountNormalized.withColumn('slen', slen(amountNormalized['Amount']))
amountNormalized_with.cache()
amountNormalized_with.show()
mmScaler = MinMaxScaler(inputCol="slen", outputCol="scaled")
model = mmScaler.fit(amountNormalized_with)
print mmScaler.explainParams()
model.transform(amountNormalized_with).show()

+------+--------+
|Amount|    slen|
+------+--------+
|149.62|[149.62]|
|  2.69|  [2.69]|
|378.66|[378.66]|
| 123.5| [123.5]|
| 69.99| [69.99]|
|  3.67|  [3.67]|
|  4.99|  [4.99]|
|  40.8|  [40.8]|
|  93.2|  [93.2]|
|  3.68|  [3.68]|
|   7.8|   [7.8]|
|  9.99|  [9.99]|
| 121.5| [121.5]|
|  27.5|  [27.5]|
|  58.8|  [58.8]|
| 15.99| [15.99]|
| 12.99| [12.99]|
|  0.89|  [0.89]|
|  46.8|  [46.8]|
|   5.0|   [5.0]|
+------+--------+
only showing top 20 rows

inputCol: input column name. (current: slen)
max: Upper bound of the output feature range (default: 1.0)
min: Lower bound of the output feature range (default: 0.0)
outputCol: output column name. (default: MinMaxScaler_4fab95417abb648294c4__output, current: scaled)
+------+--------+--------------------+
|Amount|    slen|              scaled|
+------+--------+--------------------+
|149.62|[149.62]|[0.00582379308680...|
|  2.69|  [2.69]|[1.04705276056044...|
|378.66|[378.66]|[0.01473892187040...|
| 123.5| [123.5]|[0.00480710096391...|
| 6

In [40]:
model.transform(dataframe_with).printSchema()

root
 |-- Time: decimal(10,0) (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double